In [1]:
from __future__ import division
import pymongo
import bson.json_util

In [2]:
data_path = '/Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/'
data_files = ['assigned_to',
              'bug_status',
              'cc',
              'component',
              'op_sys',
              'priority',
              'product',
              'reports',
              'resolution',
              'severity',
              'short_desc',
              'version']

In [3]:
mc = pymongo.MongoClient()
db = mc['bugs']

In [4]:
for json_file in data_files:
    json_path = data_path + json_file + '.json'
    coll = db[json_file]
    print 'processing %s...' % json_path
    with open(json_path,'r') as f:
        data = bson.json_util.loads(f.read())
        for key, val in data[json_file].iteritems():
            if type(val) == dict:
                val['id'] = key
                coll.insert_one(val)
            else: #assuming list, try to explode
                for x in val:
                    x['id'] = key
                coll.insert_many(val)

processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/assigned_to.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/bug_status.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/cc.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/component.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/op_sys.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/priority.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/product.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/reports.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/resolution.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/severity.json...
processing /Users/lucka/galvanize/msr2013-bug_dataset/data/v02/mozilla/short_desc.json...
processing /Users/lucka/galvaniz

In [5]:
#list(coll.find(limit=10))
#coll.count()